# Gradle notes

Gradle is one of the two major jvm build systems, the other major player being maven.  There are also
some other tools like sbt and leiningen, but maven and gradle are definitely the two heavy weights.

Google and Netflix choose gradle as their build tooling system.  Gradle is much more flexible and powerful
than maven, but also comes with a much higher learning curve.  These notes will go over some of the
things I have learned about gradle, and provide some background on how and why to do things.

## Gradle Basics

There are a couple of important things to understand about gradle:

- It builds a task graph (an acyclic graph of tasks)
- A task can be defined in the build script, or a plugin
- Tasks have types
- Tasks can declare other tasks that must run first before they can
- Gradle does not have a lifecycle like maven, but can emulate them
- Build scripts have a name of build.gradle[.kts] and define the configuration of a build
    - The repositories, plugins, tasks, publishing, plugin config, etc
    - Are scripts, and can contain arbitrary code (though this is frowned on)
- Plugins provide extra functionality to gradle like building java libraries or fat jars
    - In a multiproject, only the root build script can have versioned plugins
- Repositories determine where jar dependencies can be pulled from
- Dependencies have configuration types that specify not just what artifacts the projects need, but when
    - ie, implementation, compileTimeOnly, testImplementation, etc
    - Dependency management is more powerful and flexible in gradle than Maven
    - the `java-library` plugin provides implementation, runtimeOnly, etc
- Gradle looks for source files by convention, but they can be defined through `sourceSets`
- Artifacts are published to maven through a separate plugin
- Kotlin's configuration is provided by a plugin through `kotlin(jvm)`

## Creating multi projects

A gradle multiproject allows you to create several subprojects contained within another parent project.
This is useful when you have a group of related subprojects and one subproject has a dependency on one or
more subprojects.  Without a multiproject, you would need to develop your dependencies, and then either
install them locally (ie, a gradle publishLocal), or push them to a real repository.  If the dependency
subprojects are in constant flux, this can be a huge drawback.

A multiproject lets you tell gradle under the `dependencies` that one subproject has a dependency on
another subproject.  That way, gradle takes care of it for you.

### Gotchas with multiprojects

Probably the bigger gotcha with multiprojects, is that the `allprojects` configuration block is no longer
recommended by the Gradle team.  This block is useful to share configuration data between all the subprojects.
Gradle's recommended solution is to instead use custom gradle plugins.

Another gotcha with multiprojects is that CI/CD becomes more challenging.  If you make a change to only
one subproject, now everything has to be tested and redeployed.

## Gradle composite builds

I need to investigate this more, but an alternative to multiprojects is something gradle calls a composite
build.  It appears that this is a build that combines several other projects, but still keep them separate from
each other (ie, they are source controlled differently, and can run their own independent CI/CD pipelines).